In [1]:
from functools import reduce
import requests

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [3]:
from tqdm.notebook import tqdm

## Open County data

In [4]:
file_path = "data/cb_2024_us_county_500k/cb_2024_us_county_500k.shp"
counties_gdf = gpd.read_file(file_path)

## Get Ethnic Data

In [5]:
r = requests.get("https://api.census.gov/data/2023/acs/acs5/groups/B05006.json")
columns_obj = r.json()

### Get columns to query and rename for later

In [6]:
columns = []
rename_vars = {}
variables = columns_obj["variables"]
for name, variable in list(variables.items()):
    v_split = variable["label"].split("!!")
    if len(v_split) < 4:
        continue

    if v_split[0] == "Estimate":
        label = v_split[-1]
        rename_vars[name] = label

    if (name.endswith("E") or name.endswith("M")) and v_split[2] == "Asia:":
        columns.append(name)

In [7]:
dfs = []
for i in tqdm(range(0, len(columns), 49), desc="Requesting data in batches"):
    columns_with_geoid = columns[i : i + 49]
    columns_with_geoid.append("GEO_ID")
    columns_formatted = ",".join(columns_with_geoid)
    url = f"https://api.census.gov/data/2023/acs/acs5?get={columns_formatted}&for=county:*"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    dfs.append(df)

Requesting data in batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
df = reduce(lambda left, right: pd.merge(left, right, on="GEO_ID", how="inner"), dfs)

In [9]:
df = df.replace(np.nan, 0)

In [10]:
estimate_cols = [col for col in df.columns if col.endswith("E")]

In [11]:
formtted_df = df[["GEO_ID", *estimate_cols]]
formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)

/tmp/ipykernel_8991/4148564808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df[estimate_cols] = formtted_df[estimate_cols].astype(int)


In [12]:
filtered_estimate_cols = {
    k: v
    for k, v in rename_vars.items()
    if k in estimate_cols
    and (":" not in v or v == "United Kingdom (inc. Crown Dependencies):")
}

In [16]:
formtted_df["most_common_asian_origin_raw"] = formtted_df[
    list(filtered_estimate_cols.keys())
].idxmax(axis=1)

/tmp/ipykernel_8991/2230448707.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df["most_common_asian_origin_raw"] = formtted_df[


In [17]:
def check_margin_error(row) -> str:
    geo_id = row["GEO_ID"]
    ethnicity_col = row["most_common_asian_origin_raw"]
    val = row[ethnicity_col]

    if not val:
        return None

    moe_col = ethnicity_col.replace("E", "M")
    moe_val = int(df[df["GEO_ID"] == geo_id][moe_col])

    rmoe_val = abs(moe_val / val)
    if rmoe_val < 0.25:
        return variables[ethnicity_col]["label"].split("!!")[-1]
    else:
        return None

In [18]:
formtted_df["most_common_asian_origin"] = formtted_df.apply(
    lambda row: check_margin_error(row), axis=1
)

/tmp/ipykernel_8991/4230385589.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  moe_val = int(df[df["GEO_ID"] == geo_id][moe_col])
/tmp/ipykernel_8991/3029073496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formtted_df["most_common_asian_origin"] = formtted_df.apply(


In [19]:
rename_vars["GEO_ID"] = "GEOIDFQ"
formtted_df = formtted_df.rename(columns=rename_vars)

In [20]:
formtted_df.groupby("most_common_asian_origin").size().reset_index(
    name="COUNT"
).sort_values("COUNT", ascending=False)

,most_common_asian_origin,COUNT
5,India,137
10,Philippines,68
4,"China, excluding Hong Kong and Taiwan",39
11,Vietnam,30
8,Laos,9
7,Korea,7
3,Burma (Myanmar),5
1,Bangladesh,2
2,Bhutan,2
0,Afghanistan,1


## Merge Data

In [21]:
gdf = counties_gdf.merge(formtted_df, on="GEOIDFQ", how="inner")

In [22]:
gdf = gdf.to_crs(9311)
gdf.to_file("data/Citizen_Asian_Origin_Region_Per_County.gpkg")